# Data Preparation

### Import Libraries

In [160]:
import pandas as pd
import numpy as np
import seaborn as sns
import sqlite3
import matplotlib.pyplot as plt
import zipfile
import os
import ast
from sklearn.linear_model import LinearRegression
from matplotlib import style
from sklearn.model_selection import train_test_split

In [161]:
cd 

C:\Users\emmam


In [162]:
cd Documents

C:\Users\emmam\Documents


In [163]:
cd Flatiron

C:\Users\emmam\Documents\Flatiron


In [164]:
cd phase_2

C:\Users\emmam\Documents\Flatiron\phase_2


In [165]:
cd film-analytics

C:\Users\emmam\Documents\Flatiron\phase_2\film-analytics


In [166]:
cd data

C:\Users\emmam\Documents\Flatiron\phase_2\film-analytics\data


### Database Connection and Table Overview


In [167]:
conn = sqlite3.connect('im.db')

In [168]:
pd.read_sql("""SELECT name FROM sqlite_master WHERE type='table';""", conn)

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


In [169]:
imdb_movie_info = pd.read_sql("""SELECT primary_title, start_year, averagerating AS average_rating, numvotes AS num_votes, 
primary_name AS director
FROM persons
JOIN directors ON persons.person_id = directors.person_id
JOIN movie_basics ON directors.movie_id = movie_basics.movie_id
JOIN movie_ratings ON movie_basics.movie_id = movie_ratings.movie_id;""", conn)

In [170]:
imdb_movie_info.head()

,primary_title,start_year,average_rating,num_votes,director
0,Paano na kaya,2010,6.4,77,Ruel S. Bayani
1,Paano na kaya,2010,6.4,77,Ruel S. Bayani
2,Paano na kaya,2010,6.4,77,Ruel S. Bayani
3,Paano na kaya,2010,6.4,77,Ruel S. Bayani
4,No Other Woman,2011,6.4,256,Ruel S. Bayani


In [171]:
imdb_movie_info.drop_duplicates(inplace=True)

In [172]:
imdb_movie_info_grouped = imdb_movie_info.groupby('director')[['average_rating', 'num_votes']].mean()
imdb_movie_info_grouped.head()

,average_rating,num_votes
director,,
A Normale Jef,7.2,31.0
A'Ali de Sousa,4.2,55.0
A. Blaine Miller,7.0,8.0
A. Cengiz Mert,3.2,6.0
A. Fishman,7.8,37.0


### Loading Datasets

In [173]:
movie_gross = pd.read_csv('bom.movie_gross.csv.gz')
movie_info = pd.read_csv('rt.movie_info.tsv.gz', delimiter='\t')
movie_reviews = pd.read_csv('rt.reviews.tsv.gz', delimiter='\t', encoding='latin-1')
movie_votes = pd.read_csv('tmdb.movies.csv.gz')
movie_budgets = pd.read_csv('tn.movie_budgets.csv.gz')

In [174]:
def remove_special_characters(df, column_name):
    df[column_name] = df[column_name].str.replace('[^a-zA-Z0-9 ]', '', regex=True).str.lower().str.strip()
    return df

movie_gross = remove_special_characters(movie_gross, 'title')
movie_votes = remove_special_characters(movie_votes, 'title')
movie_budgets = remove_special_characters(movie_budgets, 'movie')
imdb_movie_info_grouped = remove_special_characters(imdb_movie_info, 'primary_title')

merged_data = pd.merge(movie_gross, movie_votes, how='inner', left_on='title', right_on='title')
merged_data = pd.merge(merged_data, movie_budgets, how='inner', left_on='title', right_on='movie')

df = pd.merge(imdb_movie_info, merged_data, left_on='primary_title', right_on='movie', how='inner')

df.head()

,primary_title,start_year,average_rating,num_votes,director,title,studio,domestic_gross_x,foreign_gross,year,...,popularity,release_date_x,vote_average,vote_count,id_y,release_date_y,movie,production_budget,domestic_gross_y,worldwide_gross
0,diary of a wimpy kid rodrick rules,2011,6.6,23135,David Bowers,diary of a wimpy kid rodrick rules,Fox,52700000.0,19700000,2011,...,8.961,2011-03-25,6.4,492,80,"Mar 25, 2011",diary of a wimpy kid rodrick rules,"$18,000,000","$52,698,535","$73,695,194"
1,diary of a wimpy kid dog days,2012,6.3,19571,David Bowers,diary of a wimpy kid dog days,Fox,49000000.0,28100000,2012,...,11.360,2012-08-03,6.2,440,17,"Aug 3, 2012",diary of a wimpy kid dog days,"$22,000,000","$49,008,662","$77,229,695"
2,diary of a wimpy kid the long haul,2017,4.4,5635,David Bowers,diary of a wimpy kid the long haul,Fox,20700000.0,19400000,2017,...,10.839,2017-05-19,5.6,193,27,"May 19, 2017",diary of a wimpy kid the long haul,"$22,000,000","$20,738,724","$35,609,577"
3,blue valentine,2010,7.4,170089,Derek Cianfrance,blue valentine,Wein.,9700000.0,2600000,2010,...,8.994,2010-12-29,6.9,1677,25,"Dec 29, 2010",blue valentine,"$1,000,000","$9,737,892","$16,566,240"
4,blue valentine,2010,7.4,170089,Derek Cianfrance,blue valentine,Wein.,9700000.0,2600000,2010,...,8.994,2010-12-29,6.9,1677,25,"Dec 29, 2010",blue valentine,"$1,000,000","$9,737,892","$16,566,240"


In [175]:
movie_info = pd.read_csv('rt.movie_info.tsv.gz', delimiter='\t')
movie_reviews = pd.read_csv('rt.reviews.tsv.gz', delimiter='\t', encoding='latin-1')

merged_rt_data = pd.merge(movie_info, movie_reviews, on='id', how='left')

merged_rt_data.head()

,id,synopsis,rating_x,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio,review,rating_y,fresh,critic,top_critic,publisher,date
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0.0,Patrick Nabarro,"November 10, 2018"
2,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0.0,io9.com,"May 23, 2018"
3,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0.0,Stream on Demand,"January 4, 2018"
4,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0.0,MUBI,"November 16, 2017"


### Defining Functions

In [176]:
def drop_nans(df, column_name):
    """
    Drop NaNs from specified columns from a Pandas DataFrame.

    :param df: Pandas DataFrame from which NaNs will be dropped.
    :param column_name: Target column.
    """
    df.dropna(subset=column_name, inplace=True)

In [177]:
def drop_columns(df, columns_to_drop):
    """
    Drop specified columns from a Pandas DataFrame.

    :param df: Pandas DataFrame from which columns will be dropped.
    :param columns_to_drop: List of column names to drop.
    """
    df.drop(columns=columns_to_drop, inplace=True)

In [178]:
def format_column(df, columns):
    for column in columns:
        if column in df.columns:
            # Convert the column to string, replace commas and dollar signs, then convert to float
            df[column] = df[column].astype(str).str.replace(',', '').str.replace('$', '').astype(float)
    return df

In [179]:
df.drop_duplicates(subset=['title', 'release_date_x'], inplace=True)

In [180]:
df.shape

(1346, 25)

In [181]:
drop_nans(df, ['studio', 'domestic_gross_x'])
drop_columns(df, ['foreign_gross', 'Unnamed: 0', 'id_x', 'original_language', 'original_title', 'id_y', 
                           'release_date_y', 'domestic_gross_y'])
format_column(df, ['domestic_gross_x', 'worldwide_gross', 'production_budget'])

,primary_title,start_year,average_rating,num_votes,director,title,studio,domestic_gross_x,year,genre_ids,popularity,release_date_x,vote_average,vote_count,movie,production_budget,worldwide_gross
0,diary of a wimpy kid rodrick rules,2011,6.6,23135,David Bowers,diary of a wimpy kid rodrick rules,Fox,52700000.0,2011,"[10751, 35]",8.961,2011-03-25,6.4,492,diary of a wimpy kid rodrick rules,18000000.0,73695194.0
1,diary of a wimpy kid dog days,2012,6.3,19571,David Bowers,diary of a wimpy kid dog days,Fox,49000000.0,2012,"[35, 10751]",11.360,2012-08-03,6.2,440,diary of a wimpy kid dog days,22000000.0,77229695.0
2,diary of a wimpy kid the long haul,2017,4.4,5635,David Bowers,diary of a wimpy kid the long haul,Fox,20700000.0,2017,"[35, 10751]",10.839,2017-05-19,5.6,193,diary of a wimpy kid the long haul,22000000.0,35609577.0
3,blue valentine,2010,7.4,170089,Derek Cianfrance,blue valentine,Wein.,9700000.0,2010,"[18, 10749]",8.994,2010-12-29,6.9,1677,blue valentine,1000000.0,16566240.0
5,the place beyond the pines,2012,7.3,226765,Derek Cianfrance,the place beyond the pines,Focus,21400000.0,2013,"[18, 80]",12.960,2013-03-27,6.9,2781,the place beyond the pines,15000000.0,47011449.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2191,whiplash,2014,8.5,616916,Damien Chazelle,whiplash,SPC,13100000.0,2014,"[18, 10402]",6.298,2013-01-18,7.9,94,whiplash,3300000.0,38969037.0
2192,whiplash,2014,8.5,616916,Damien Chazelle,whiplash,SPC,13100000.0,2014,"[18, 10402]",28.784,2014-10-10,8.4,7908,whiplash,3300000.0,38969037.0
2193,la la land,2016,8.0,436070,Damien Chazelle,la la land,LG/S,151100000.0,2016,"[35, 18, 10749]",16.275,2016-12-09,7.9,10028,la la land,20000000.0,426351163.0
2195,hitman agent 47,2015,5.7,80510,Aleksander Bach,hitman agent 47,Fox,22500000.0,2015,"[28, 80, 53]",14.082,2015-08-21,5.6,1764,hitman agent 47,35000000.0,82182803.0


In [182]:
df['studio'] = df['studio'].str.strip().str.strip('.').str.upper()

In [183]:
df.rename(columns={'domestic_gross_x': 'domestic_gross'}, inplace=True)
df.rename(columns={'release_date_x': 'release_date'}, inplace=True)

In [184]:
df.columns

Index(['primary_title', 'start_year', 'average_rating', 'num_votes',
       'director', 'title', 'studio', 'domestic_gross', 'year', 'genre_ids',
       'popularity', 'release_date', 'vote_average', 'vote_count', 'movie',
       'production_budget', 'worldwide_gross'],
      dtype='object')

In [185]:
movie_genres = {
  "genres": [
    {
      "id": 28,
      "name": "action"
    },
    {
      "id": 12,
      "name": "adventure"
    },
    {
      "id": 16,
      "name": "animation"
    },
    {
      "id": 35,
      "name": "comedy"
    },
    {
      "id": 80,
      "name": "crime"
    },
    {
      "id": 99,
      "name": "documentary"
    },
    {
      "id": 18,
      "name": "drama"
    },
    {
      "id": 10751,
      "name": "family"
    },
    {
      "id": 14,
      "name": "fantasy"
    },
    {
      "id": 36,
      "name": "history"
    },
    {
      "id": 27,
      "name": "horror"
    },
    {
      "id": 10402,
      "name": "music"
    },
    {
      "id": 9648,
      "name": "mystery"
    },
    {
      "id": 10749,
      "name": "romance"
    },
    {
      "id": 878,
      "name": "science fiction"
    },
    {
      "id": 10770,
      "name": "tv movie"
    },
    {
      "id": 53,
      "name": "thriller"
    },
    {
      "id": 10752,
      "name": "war"
    },
    {
      "id": 37,
      "name": "western"
    }
  ]
}    

In [186]:
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
import ast

# Assuming movie_genres and df are already defined and genre_mapping is created
genre_mapping = {int(genre['id']): genre['name'] for genre in movie_genres['genres']}

# Convert genre_ids to lists of integers
df['genre_ids'] = df['genre_ids'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Initialize the binarizer
mlb = MultiLabelBinarizer()

# Fit and transform the genre_ids column
genre_binarized = mlb.fit_transform(df['genre_ids'])

# Correctly map columns to genre names
valid_genre_ids = [int(col) for col in mlb.classes_]
genre_names = [genre_mapping[col] for col in valid_genre_ids]

# Create a new DataFrame with the binarized genres and correct column names
genre_df = pd.DataFrame(genre_binarized, columns=genre_names)

# Add title and release_date columns to genre_df
genre_df['title'] = df['title']
genre_df['release_date'] = df['release_date']

# Merge the binarized genres with the original DataFrame
df = pd.concat([df, genre_df], axis=1)

# Drop the original genre_ids column if it's no longer needed
df.drop('genre_ids', axis=1, inplace=True)

# Display the new DataFrame
df.head()

,primary_title,start_year,average_rating,num_votes,director,title,studio,domestic_gross,year,popularity,...,documentary,science fiction,mystery,music,romance,family,war,tv movie,title,release_date
0,diary of a wimpy kid rodrick rules,2011.0,6.6,23135.0,David Bowers,diary of a wimpy kid rodrick rules,FOX,52700000.0,2011.0,8.961,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,diary of a wimpy kid rodrick rules,2011-03-25
1,diary of a wimpy kid dog days,2012.0,6.3,19571.0,David Bowers,diary of a wimpy kid dog days,FOX,49000000.0,2012.0,11.360,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,diary of a wimpy kid dog days,2012-08-03
2,diary of a wimpy kid the long haul,2017.0,4.4,5635.0,David Bowers,diary of a wimpy kid the long haul,FOX,20700000.0,2017.0,10.839,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,diary of a wimpy kid the long haul,2017-05-19
3,blue valentine,2010.0,7.4,170089.0,Derek Cianfrance,blue valentine,WEIN,9700000.0,2010.0,8.994,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,blue valentine,2010-12-29
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [28]:
director_ratings = df.groupby('director')['vote_average'].mean().reset_index()
director_ratings.columns = ['director', 'average_rating']

director_ratings['rating_category'] = pd.qcut(director_ratings['average_rating'], 5, labels=['Bottom Tier', 'Low Tier', 
                                                                                             'Mid Tier', 'High Tier', 'Top Tier'])


df_encoded = df.merge(director_ratings, on='director', how='left')

In [187]:
df['director'] = df['director'].astype(str)

In [188]:
from sklearn.preprocessing import LabelEncoder

# Create a label encoder
encoder = LabelEncoder()

# Encode the 'director' column in your DataFrame
df['director_encoded'] = encoder.fit_transform(df['director'])

In [189]:
df_encoded['average_rating_y'].value_counts()

6.50    51
6.10    48
6.20    45
6.60    45
6.00    41
        ..
7.65     2
8.30     1
4.30     1
4.10     1
3.10     1
Name: average_rating_y, Length: 162, dtype: int64

In [190]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Assuming df is already defined

# Replace NaN values with a placeholder if necessary
df['director'] = df['director'].fillna('Unknown')

# Ensure all values in the 'director' column are strings
df['director'] = df['director'].astype(str)

# Initialize the encoder
encoder = LabelEncoder()

# Encode the 'director' column
df['director_encoded'] = encoder.fit_transform(df['director'])

# Display the new DataFrame
df[['director', 'director_encoded']].head()

,director,director_encoded
0,David Bowers,167
1,David Bowers,167
2,David Bowers,167
3,Derek Cianfrance,203
4,nan,828


### Data Normalization

In [192]:
df.dtypes

primary_title         object
start_year           float64
average_rating       float64
num_votes            float64
director              object
title                 object
studio                object
domestic_gross       float64
year                 float64
popularity           float64
release_date          object
vote_average         float64
vote_count           float64
movie                 object
production_budget    float64
worldwide_gross      float64
adventure            float64
fantasy              float64
animation            float64
drama                float64
horror               float64
action               float64
comedy               float64
history              float64
western              float64
thriller             float64
crime                float64
documentary          float64
science fiction      float64
mystery              float64
music                float64
romance              float64
family               float64
war                  float64
tv movie      